In [1]:
import os
import numpy as np
import json
import pandas as pd

In [2]:
tasks = [
    "agieval", "boolq", "hellaswag", "mathqa", "mrpc", "piqa", "qnli", "rte", "social_iqa", "winogrande",
    "arc_challenge", "copa", "lambada_standard", "mmlu", "multirc", "pubmedqa", "qqp", "sciq", "sst2", "wnli",
    "arc_easy", "gsm8k", "logiqa", "mnli", "openbookqa", "qasper", "race", "scrolls", "wikitext", "wsc"
]


In [13]:
all_dfs = []
nshot = 0
for task_name in tasks:
    task_performance = {}
    for ds_version in ['cc', 'cc_merged_v1', 'cc_merged_v2', 'cc_merged_v3']:
        # BASE_PATH = f"/home/aiops/zhuty/tinyllama/scripts/lm_harness_eval/out/home/aiops/zhuty/tinyllama/out/tiny_LLaMA_1b_8k_{ds_version}_8k/iter-380000-ckpt-step-47500_hf"
        BASE_PATH = f"/home/aiops/zhuty/tinyllama/scripts/lm_harness_eval/out/home/aiops/zhuty/lm_indexer_data/tyzhu/flan_max_300_added_tyzhu_tiny_LLaMA_1b_8k_{ds_version}_8k_iter-380000-ckpt-step-47500_hf/checkpoint-12387"
        result_file_path = os.path.join(BASE_PATH, f"{task_name}", "results.json")
        if not os.path.exists(result_file_path):
            print(f"File not found", task_name, ds_version)
            continue
        result = json.load(open(result_file_path))
        # print(result)
        assert result['n-shot'][task_name] == 0
        result = result['results'][task_name]
        task_performance[ds_version] = result
        # break
    # make the dataframe
    print("Task:", task_name)
    if len(task_performance) == 0:
        continue
    df = pd.DataFrame(task_performance)
    df = pd.concat([df.loc[['alias']], df.drop('alias')])
    all_dfs.append(df)
    print(df)

File not found agieval cc
File not found agieval cc_merged_v1
File not found agieval cc_merged_v2
File not found agieval cc_merged_v3
Task: agieval
Task: boolq
                       cc cc_merged_v1 cc_merged_v2 cc_merged_v3
alias               boolq        boolq        boolq        boolq
acc,none         0.589602      0.58104      0.61682     0.605199
acc_stderr,none  0.008603     0.008629     0.008503     0.008549
Task: hellaswag
                             cc cc_merged_v1 cc_merged_v2 cc_merged_v3
alias                 hellaswag    hellaswag    hellaswag    hellaswag
acc,none               0.332304     0.340271      0.33579     0.340968
acc_stderr,none        0.004701     0.004728     0.004713     0.004731
acc_norm,none          0.396734     0.405397     0.409281     0.411173
acc_norm_stderr,none   0.004882       0.0049     0.004907      0.00491
Task: mathqa
                            cc cc_merged_v1 cc_merged_v2 cc_merged_v3
alias                   mathqa       mathqa       mathq

In [14]:
new_df = pd.concat(all_dfs, axis=0)

In [15]:
new_df

,cc,cc_merged_v1,cc_merged_v2,cc_merged_v3
alias,boolq,boolq,boolq,boolq
"acc,none",0.589602,0.58104,0.61682,0.605199
"acc_stderr,none",0.008603,0.008629,0.008503,0.008549
alias,hellaswag,hellaswag,hellaswag,hellaswag
"acc,none",0.332304,0.340271,0.33579,0.340968
...,...,...,...,...
"bits_per_byte,none",0.857763,0.848877,1.415619,0.850997
"bits_per_byte_stderr,none",N/A,N/A,N/A,N/A
alias,wsc,wsc,wsc,wsc
"acc,none",0.471154,0.375,0.394231,0.365385
